In [2]:
import numpy as np
import nnfs
from nnfs.datasets import spiral_data

nnfs.init()


In [3]:
class Linear():
    def __init__(self,inputs,outputs):
        self.weights=0.01*np.random.randn(outputs,inputs)
        self.biases=np.zeros((1,outputs))

    def forward(self,inputs):
        self.inputs=inputs
        self.outputs=np.dot(inputs,self.weights.T)+self.biases
        return self.outputs
    
    def backward(self,dnextinputs):
        self.dweights=np.dot(dnextinputs.T,self.inputs)
        self.dbiases=np.sum(dnextinputs,axis=0,keepdims=True)
        self.dinputs=np.dot(dnextinputs,self.weights)
        return self.dinputs

In [4]:
class Relu():
    def __init__(self) -> None:
        pass
    
    def forward(self,inputs):
        self.inputs=inputs
        self.outputs=np.maximum(0,inputs)
        return self.outputs


    def backward(self,dnextinputs):
        self.dinputs=dnextinputs
        self.dinputs[self.inputs<=0]=0
        return self.dinputs



    

In [5]:
class Softmax():
    def __init__(self) -> None:
        pass

    def forward(self,inputs):
        self.inputs=inputs
        exp=np.exp(inputs -np.max(inputs, axis=1,keepdims=True))
        exp_sum=np.sum(exp,axis=1,keepdims=True)
        self.outputs=exp/exp_sum
        return self.outputs

    def backwards(self,dnextinputs):
        self.dinputs = np.zeros_like(dnextinputs)
        i=0
        for output, dnextinput in zip(self.outputs, dnextinputs):
            output = output.reshape(-1, 1)
            Jacobian= np.diagflat(output) - np.dot(output,output.T)
            self.dinputs[i] = np.dot(Jacobian, dnextinput)
            i+=1
        return self.dinputs


In [6]:
class CrossEntropyLoss():
    def __init__(self) -> None:
        pass
        
    def forward(self,inputs,labels):
        self.labels=labels
        self.inputs=inputs
        epsilon=1e-7
        inputs=np.clip(inputs,epsilon,1-epsilon)
        one_hot=np.eye(len(inputs[0]))[labels]
        self.output=-np.sum(one_hot*np.log(inputs))/one_hot.shape[0]
        return self.output
    
    def backward(self):
        one_hot=np.eye(len(self.inputs[0]))[self.labels]
        self.dinputs=(-one_hot/self.inputs)/one_hot.shape[0]
        return self.dinputs


class Optimizer_Adam():
    def __init__(self,learning_rate):
        self.learning_rate=learning_rate
        self.current_learning_rate=learning_rate
        self.learning_rate_step

In [7]:
a=np.array([2,3,4,1,2])
b=np.array([3,4,7,-1,3])
c=a-b
c.shape[0]


5

In [8]:
X, y = spiral_data(samples=100, classes=3)
X=np.array([[0.        , 0.        ],
 [0.00299556, 0.00964661],
 [0.01288097, 0.01556285],
 [0.02997479, 0.0044481 ],
 [0.03931246 ,0.00932828]])
y=np.array([0, 0 ,2 ,0 ,0])

dense1 = Linear(2, 3)

activation1 =Relu()


dense2 = Linear(3, 3)

activation2 =Softmax()
lossFun=CrossEntropyLoss()



x=dense1.forward(X)
x=activation1.forward(x)
x=dense2.forward(x)
x=activation2.forward(x)
loss=lossFun.forward(x,y)

print("loss",loss)
predictions = np.argmax(x, axis=1)
accuracy = np.mean(predictions==y)
print('acc:', accuracy)


# # Backward pass
z=lossFun.backward()
z=activation2.backwards(z)
z=dense2.backward(z)
z=activation1.backward(z)
z=dense1.backward(z)


# Print gradients
print("d1w",dense1.dweights.T)
print("d1b",dense1.dbiases)
print("d2w",dense2.dweights.T)
print("d2b",dense2.dbiases)

loss 1.0986122131347655
acc: 0.2
d1w [[ 9.2849623e-06  0.0000000e+00 -1.0265569e-05]
 [ 1.6771819e-05  0.0000000e+00  1.9986226e-05]]
d1b [[0.00142796 0.         0.00076794]]
d2w [[-1.0124815e-05  1.4038321e-05 -3.9135052e-06]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00]
 [-5.2157357e-05  2.7492268e-05  2.4665092e-05]]
d2b [[-0.46666667  0.3333336   0.13333309]]


In [9]:
import pandas as pd

In [10]:
data.head(5)

NameError: name 'data' is not defined

In [ ]:
  
categorical_data=["Sex","Existence_of_diphasic_derivation_of_P_wave","Existence_of_diphasic_derivation_of_R_wave",
                  "Existence_of_diphasic_derivation_of_T_wave","Existence_of_ragged_P_wave",
                  "Existence_of_ragged_R_wave","Existence_of_ragged_T_wave"]


# check empty/valid data
# remove/change them or columns
#check caterogical data (0,1) or higher if higher get dummies
# normalize data
# check correlation
# remove data with low corelation
# wykład 2 do  
# 1-6 pozad drugim wchodza do 1 kolokwia
#

